Load dataframes with acf and calculate tau

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit, OptimizeWarning
from sklearn.metrics import r2_score
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

### Get the data

In [2]:
results_folder = 'E:\\projects_q_30_10_2024\\isttc\\results\\monkey\\fixation_period_1000ms\\'
save_folder = 'E:\\projects_q_30_10_2024\\isttc\\results\\monkey\\fixation_period_1000ms\\'

### Calculate tau

In [3]:
def func_single_exp(x, a, b, c):
    """
    Exponential function to fit the data.
    :param x: 1d array, independent variable
    :param a: float, parameter to fit
    :param b: float, parameter to fit
    :param c: float, parameter to fit
    :return: callable
    """
    #return a * np.exp(-b * x) + c
    return a * (np.exp(-b * x) + c) # as in the paper


def fit_single_exp(ydata_to_fit_, start_idx_=1):
    """
    Fit function func_exp to data using non-linear least square.

    todo check that - important point: Fit is done from the first ACF value (acf[0] is skipped, it is done like this
    in the papers, still not sure)

    :param ydata_to_fit_: 1d array, the dependant data to fit
    :param start_idx_: int, index to start fitting from
    :return: fit_popt, fit_pcov, tau, fit_r_squared
    """
    t = np.linspace(0, len(ydata_to_fit_)-1, len(ydata_to_fit_)).astype(int)

    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        try:
            popt, pcov = curve_fit(func_single_exp, t[start_idx_:], ydata_to_fit_[start_idx_:], maxfev=1000000000) # I used 5000, now it is like in Siegle
            fit_popt = popt
            fit_pcov = pcov
            tau = 1 / fit_popt[1]
            # fit r-squared
            y_pred = func_single_exp(t[start_idx_:], *popt)
            fit_r_squared = r2_score(ydata_to_fit_[start_idx_:], y_pred)
        except RuntimeError as e:
            print('RuntimeError: {}'. format(e))
            fit_popt, fit_pcov, tau, fit_r_squared = np.nan, np.nan, np.nan, np.nan
        except OptimizeWarning as o:
            print('OptimizeWarning: {}'. format(o))
            fit_popt, fit_pcov, tau, fit_r_squared = np.nan, np.nan, np.nan, np.nan
        except RuntimeWarning as re:
            print('RuntimeWarning: {}'. format(re))
            fit_popt, fit_pcov, tau, fit_r_squared = np.nan, np.nan, np.nan, np.nan
        except ValueError as ve:
            print('ValueError: {}'. format(ve))
            print('Possible reason: acf contains NaNs, low spike count')
            fit_popt, fit_pcov, tau, fit_r_squared = np.nan, np.nan, np.nan, np.nan

    return fit_popt, fit_pcov, tau, fit_r_squared

In [4]:
def calc_tau(acf_df_, acf_cols_):
    acf_2d = acf_df_[acf_cols_].values
    acf_2d = acf_2d.astype(np.float64) # RuntimeWarning: overflow encountered in matmul sttc start_idx 2
    n_units = acf_2d.shape[0]
    print('Calculating taus for {}'.format(acf_2d.shape))
    
    fit_popt_a_l, fit_popt_b_l, fit_popt_c_l = [],[],[]
    fit_tau_l = []
    fit_r_squared_l = []
    
    for i in range(n_units):
        fit_popt, fit_pcov, tau, fit_r_squared = fit_single_exp(acf_2d[i,:], start_idx_=2)
        if  type(fit_popt) == np.ndarray:
            fit_popt_a_l.append(fit_popt[0])
            fit_popt_b_l.append(fit_popt[1])
            fit_popt_c_l.append(fit_popt[2])
        else:
            fit_popt_a_l.append(np.nan)
            fit_popt_b_l.append(np.nan)
            fit_popt_c_l.append(np.nan)
        fit_tau_l.append(tau)
        fit_r_squared_l.append(fit_r_squared)
    
    data_df = np.vstack((fit_popt_a_l, fit_popt_b_l, fit_popt_c_l, fit_tau_l, fit_r_squared_l)).T
    tau_df = pd.DataFrame(data_df, columns=['fit_a', 'fit_b', 'fit_c','tau', 'r_squared'])
    tau_df.insert(0, 'unit_id', acf_df_['unit_id'].values)
    # tau_df.insert(1, 'trial_id', acf_proper_df['trial_id'].values)
    tau_df['tau_ms'] = tau_df['tau'] * 50

    return tau_df

In [5]:
n_lags = 20
acf_cols = ['acf_' + str(i) for i in range(n_lags)]

In [7]:
# # pfdl pearsonr
# pfdl_acf_pearsonr_df = pd.read_pickle(results_folder + '\\binned\\pfdl\\acf\\acf_average_trial_pearsonr_with_empty_50ms_20lags_df.pkl')
# pfdl_tau_pearsonr_df = calc_tau(pfdl_acf_pearsonr_df, acf_cols)
# pfdl_tau_pearsonr_df.to_pickle(save_folder + '\\binned\\pfdl\\taus\\tau_acf_average_trial_pearsonr_with_empty_50ms_20lags_ind2_df.pkl')
# pfdl_tau_pearsonr_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

# # pfp pearsonr
# pfp_acf_pearsonr_df = pd.read_pickle(results_folder + '\\binned\\pfp\\acf\\acf_average_trial_pearsonr_with_empty_50ms_20lags_df.pkl')
# pfp_tau_pearsonr_df = calc_tau(pfp_acf_pearsonr_df, acf_cols)
# pfp_tau_pearsonr_df.to_pickle(save_folder + '\\binned\\pfp\\taus\\tau_acf_average_trial_pearsonr_with_empty_50ms_20lags_ind2_df.pkl')
# pfp_tau_pearsonr_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

# # pfdl sttc
# pfdl_acf_sttc_df = pd.read_pickle(results_folder + '\\non_binned\\pfdl\\acf\\acf_average_trial_sttc_with_empty_50ms_20lags_df.pkl')
# pfdl_tau_sttc_df = calc_tau(pfdl_acf_sttc_df, acf_cols)
# pfdl_tau_sttc_df.to_pickle(save_folder + '\\non_binned\\pfdl\\taus\\tau_acf_average_trial_sttc_with_empty_50ms_20lags_ind2_df.pkl')
# pfdl_tau_sttc_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

# pfp sttc
pfp_acf_sttc_df = pd.read_pickle(results_folder + '\\non_binned\\pfp\\acf\\acf_average_trial_sttc_with_empty_50ms_20lags_df.pkl')
pfp_tau_sttc_df = calc_tau(pfp_acf_sttc_df, acf_cols)
pfp_tau_sttc_df.to_pickle(save_folder + '\\non_binned\\pfp\\taus\\tau_acf_average_trial_sttc_with_empty_50ms_20lags_ind2_df.pkl')
pfp_tau_sttc_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

Calculating taus for (543, 20)


,unit_id,fit_a,fit_b,fit_c,tau,r_squared,tau_ms
count,366.000000,3.660000e+02,366.000000,3.660000e+02,366.000000,3.660000e+02,366.000000
mean,272.453552,4.541156e+08,2.754022,4.419601e-01,2.944267,3.097282e-01,147.213355
std,155.050365,6.017814e+09,3.862419,1.242439e+00,3.609121,3.044355e-01,180.456064
min,0.000000,-1.543382e+07,0.051376,-4.680772e+00,0.064321,-3.259339e-08,3.216071
25%,134.750000,1.121064e-01,0.243270,6.042019e-10,0.247955,1.875006e-02,12.397727
50%,281.500000,2.375919e-01,0.611998,1.059107e-01,1.633998,2.182379e-01,81.699893
75%,400.750000,5.531468e-01,4.048384,5.682172e-01,4.110664,5.642296e-01,205.533224
max,542.000000,8.300282e+10,15.546920,1.705566e+01,19.464211,9.986943e-01,973.210534
